In [1]:
import numpy as np

path_prefix = "./all/data/"
word_to_index = {} # map word to index
g_word_list = [] # sorted by frequence low to high
total_word_dict = {}
skip_words = set(['', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', 'as', 'an', 'and', 'are', 'at', 'by', 'down', 'for', 'from', 'front', 'has', 'in', 'is', 'it', 'near', 'next', 'of', 'on', 'out', 'over', 'several', 'sits', 'some', 'that', 'the', 'there', 'through', 'to', 'top', 'up', 'while', 'with'])
train_des = [] #0/1/2 array of word
test_des = [] #0/1/2 array of word
train_feature_list = [] # list of feature_array by index
test_feature_list = []
all_predict_test_des = []
train_g_category_tag = []
train_g_item_tag = []
test_g_category_tag = []
test_g_item_tag = []
MAX_INPUT = 10000
MAX_OUTPUT = 2000
COMPONENTS = 400 # need to run again
PCA_NUMBER = 2000 # need to run again
ITER = 500

In [2]:
def read_file_to_string(file_name, keep_colon):
    f = open(file_name, "r")
    content = f.read()
    f.close()
    content = content.lower()
    content = content.replace(",", " ").replace(".", " ").replace("'", " ").replace("\"", " ")
    if (keep_colon == False):
        content = content.replace(":", " ")
    return content

def read_file_to_total_word_dict(file_name, keep_colon):
    word_list = read_file_to_string(file_name, keep_colon).split()
    for name in word_list:
        if name in skip_words:
            continue
        if name in total_word_dict:
            total_word_dict[name] += 1
            continue
        else:
            total_word_dict[name] = 1
            continue

def print_word_in_array(a):
    ret = ""
    for i in range(len(a)):
        if a[i] != 0:
            ret = ret + str(a[i]) + g_word_list[i] + " "
    print(ret)

In [3]:
total_word_dict = {}
for i in range(MAX_INPUT):
    read_file_to_total_word_dict(path_prefix+"descriptions_train/"+str(i)+".txt", False)
    read_file_to_total_word_dict(path_prefix+"tags_train/"+str(i)+".txt", True)

In [4]:
import functools
def cmp(a, b):
    a_num = abs(float(a.split("___")[0]))
    b_num = abs(float(b.split("___")[0]))
    if (a_num < b_num):
        return -1
    return 1

In [5]:
# load g_word_list
g_word_list = []
word_to_index = {}
tmp_list = []
#TODO: maybe try filter on number
for word in total_word_dict:
    if total_word_dict[word] <= 0:
        continue
    tmp_list.append(str(total_word_dict[word])+"___"+word)

tmp_list = sorted(tmp_list, key=functools.cmp_to_key(cmp))

for i in range(len(tmp_list)):
    word = tmp_list[i].split("___")[1]
    g_word_list.append(word)
    word_to_index[word] = i

In [6]:
# read to tags
train_g_category_tag = [0] * MAX_INPUT
train_g_item_tag = [0] * MAX_INPUT
test_g_category_tag = [0] * MAX_OUTPUT
test_g_item_tag = [0] * MAX_OUTPUT

#TODO: add tags in descriptions
def is_in_tag(file_number, word, train):
    if train:
        return (word in train_g_category_tag[file_number]) or (word in train_g_item_tag[file_number])
    return (word in test_g_category_tag[file_number]) or (word in test_g_item_tag[file_number])

def read_tags(file_name, train):
    lines = read_file_to_string(file_name, True).split("\n")
    index = int(file_name.split("/")[-1].replace(".txt", ""))
    current_category_set = set([])
    current_item_set = set([])
    
    for line in lines:
        cat = line.split(":")[0].strip()
        if cat in word_to_index:
            current_category_set.add(cat)

        item = ''
        if (cat.strip() != line.replace(":", "").strip()):
            item = line.split(":")[1].strip()
        if item in word_to_index:
            current_item_set.add(item)
    if (train):
        train_g_category_tag[index] = current_category_set
        train_g_item_tag[index] = current_item_set
    else:
        test_g_category_tag[index] = current_category_set
        test_g_item_tag[index] = current_item_set
    

for i in range(MAX_INPUT):
    read_tags(path_prefix+"tags_train/"+str(i)+".txt", True)
for i in range(MAX_OUTPUT):
    read_tags(path_prefix+"tags_test/"+str(i)+".txt", False)

In [7]:
# where to make the change
# todo: remove count to match test output
# lenpi ci, yaoshi you dehua, jianfen
# also try norm

# count_max_weight = 2.0
# uniqueness_max_weight = 2.0
# TODO: add words in tag_item to pic
def my_weight_func(index, count, train, file_num):
    if (count <= 0):
        return 0
#     count_apply = 0
#     if is_in_tag(file_num, g_word_list[index], train):
#         count_apply = 1
#     elif count >= 3:
#         count_apply = 1
#     return 1 + (len(g_word_list)-index)*uniqueness_max_weight/len(g_word_list) + count_apply*count_max_weight

#     if count >= 5:
#         return count
#     if is_in_tag(file_num, g_word_list[index], train):
#         return 5
    return 1

def weight_occurance_array(occurance_array, train, file_num):
    ret = [0] * len(occurance_array)
    for index in range(len(occurance_array)):
        ret[index] = my_weight_func(index, occurance_array[index], train, file_num)
    return ret

def read_des_to_list(path, file_num):
    ret = []
    train = (path.find("train") != -1)
    for i in range(file_num):
        int_array = [0] * len(g_word_list)
        temp_list = read_file_to_string(path+str(i)+".txt", False).split()
        for word in temp_list:
            if not word in word_to_index:
                continue
            int_array[word_to_index[word]] += 1
        ret.append(weight_occurance_array(int_array, train, i))
    return ret

train_des = read_des_to_list(path_prefix+"descriptions_train/", MAX_INPUT)
test_des = read_des_to_list(path_prefix+"descriptions_test/", MAX_OUTPUT)

In [8]:
# add tag to desc list

#TODO: careful with tag number
def add_tag_to_train_des():
    for sentence_index in range(MAX_INPUT):
        array_to_add = [0] * len(g_word_list)
        for tag in train_g_item_tag[sentence_index]:
            array_to_add[word_to_index[tag]] = 1
        
        ret = train_des[sentence_index].copy()
        for word_index in range(len(g_word_list)):
            ret[word_index] = max(ret[word_index], array_to_add[word_index])
        train_des[sentence_index] = ret

def add_tag_to_test_des():
    for sentence_index in range(MAX_OUTPUT):
        array_to_add = [0] * len(g_word_list)
        for tag in test_g_item_tag[sentence_index]:
            array_to_add[word_to_index[tag]] = 1
        
        ret = all_predict_test_des[sentence_index].copy()
        print(all_predict_test_des[sentence_index])
        for word_index in range(len(g_word_list)):
            ret[word_index] = max(ret[word_index], array_to_add[word_index])
        all_predict_test_des[sentence_index] = ret


In [9]:
add_tag_to_train_des()

In [10]:
# load feature list
def load_feature_into_list(filename):
    f = open(filename, "r")
    content = f.readlines()
    f.close()
    train = filename.find("train")
    for line in content:
        name_index = int(line.split(".jpg,")[0].split("/")[1])
        feature_list = line.split(".jpg,")[1].split(",")
        if train != -1:
            if name_index >= MAX_INPUT:
                continue
            train_feature_list[name_index] = feature_list
        else:
            if name_index >= MAX_OUTPUT:
                continue
            test_feature_list[name_index] = feature_list

train_feature_list = [0] * MAX_INPUT
test_feature_list = [0] * MAX_OUTPUT
# load_feature_into_list(path_prefix+"features_train/features_resnet1000_train.csv")
# load_feature_into_list(path_prefix+"features_test/features_resnet1000_test.csv")
load_feature_into_list(path_prefix+"features_train/features_resnet1000intermediate_train.csv")
load_feature_into_list(path_prefix+"features_test/features_resnet1000intermediate_test.csv")

train_feature_list = np.asarray(train_feature_list, dtype=np.float64)
test_feature_list = np.asarray(test_feature_list, dtype=np.float64)

In [11]:
# from sklearn.decomposition import PCA
# pca = PCA(n_components = PCA_NUMBER)
# pca.fit(train_des)
# train_des_pca = pca.transform(train_des)
# test_des_pca = pca.transform(test_des)
train_des_pca = np.asarray(train_des)
test_des_pca = np.asarray(test_des)

print(train_des_pca.shape)
print(test_des_pca.shape)

# pca2 = PCA(n_components = PCA_NUMBER)
# pca2.fit(train_feature_list)
# train_feature_list = pca2.transform(train_feature_list)
# test_feature_list = pca2.transform(test_feature_list)
train_feature_list = np.asarray(train_feature_list)
test_feature_list = np.asarray(test_feature_list)

print(train_feature_list.shape)
print(test_feature_list.shape)

# from sklearn.cross_decomposition import PLSRegression
# print("Done 0")
# model = PLSRegression(COMPONENTS, max_iter=ITER)
# print("Done 1")
# model.fit(train_feature_list, train_des_pca)
# print("Done 2")

from sklearn.neural_network import MLPRegressor
clf = MLPRegressor(hidden_layer_sizes=(2048,1024,512), max_iter=500, solver='sgd', verbose=True)
clf.fit(train_feature_list, train_des_pca)


(10000, 9900)
(2000, 9900)
(10000, 2048)
(2000, 2048)
Iteration 1, loss = 0.00243751
Iteration 2, loss = 0.00225505
Iteration 3, loss = 0.00222952
Iteration 4, loss = 0.00222107
Iteration 5, loss = 0.00221828
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.


MLPRegressor(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(2048, 1024, 512), learning_rate='constant',
       learning_rate_init=0.001, max_iter=500, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='sgd', tol=0.0001, validation_fraction=0.1,
       verbose=True, warm_start=False)

In [12]:
print("Start 3")
all_predict_test_des = clf.predict(test_feature_list)
print("Done 3")

Start 3
Done 3


In [13]:
add_tag_to_test_des()

[-7.09746467e-05 -3.42835237e-04  1.72439592e-03 ...  2.47948722e-01
 -5.55532540e-04  2.03250677e-01]
[-7.09746467e-05 -3.42835237e-04  1.72439592e-03 ...  2.47948722e-01
 -5.55532540e-04  2.03250677e-01]
[-7.09746467e-05 -3.42835237e-04  1.72439592e-03 ...  2.47948722e-01
 -5.55532540e-04  2.03250677e-01]
[-7.09746467e-05 -3.42835237e-04  1.72439592e-03 ...  2.47948722e-01
 -5.55532540e-04  2.03250677e-01]
[-7.09746467e-05 -3.42835237e-04  1.72439592e-03 ...  2.47948722e-01
 -5.55532540e-04  2.03250677e-01]
[-7.09746467e-05 -3.42835237e-04  1.72439592e-03 ...  2.47948722e-01
 -5.55532540e-04  2.03250677e-01]
[-7.09746467e-05 -3.42835237e-04  1.72439592e-03 ...  2.47948722e-01
 -5.55532540e-04  2.03250677e-01]
[-7.09746467e-05 -3.42835237e-04  1.72439592e-03 ...  2.47948722e-01
 -5.55532540e-04  2.03250677e-01]
[-7.09746467e-05 -3.42835237e-04  1.72439592e-03 ...  2.47948722e-01
 -5.55532540e-04  2.03250677e-01]
[-7.09746467e-05 -3.42835237e-04  1.72439592e-03 ...  2.47948722e-01
 -5.

[-7.09746467e-05 -3.42835237e-04  1.72439592e-03 ...  2.47948722e-01
 -5.55532540e-04  2.03250677e-01]
[-7.09746467e-05 -3.42835237e-04  1.72439592e-03 ...  2.47948722e-01
 -5.55532540e-04  2.03250677e-01]
[-7.09746467e-05 -3.42835237e-04  1.72439592e-03 ...  2.47948722e-01
 -5.55532540e-04  2.03250677e-01]
[-7.09746467e-05 -3.42835237e-04  1.72439592e-03 ...  2.47948722e-01
 -5.55532540e-04  2.03250677e-01]
[-7.09746467e-05 -3.42835237e-04  1.72439592e-03 ...  2.47948722e-01
 -5.55532540e-04  2.03250677e-01]
[-7.09746467e-05 -3.42835237e-04  1.72439592e-03 ...  2.47948722e-01
 -5.55532540e-04  2.03250677e-01]
[-7.09746467e-05 -3.42835237e-04  1.72439592e-03 ...  2.47948722e-01
 -5.55532540e-04  2.03250677e-01]
[-7.09746467e-05 -3.42835237e-04  1.72439592e-03 ...  2.47948722e-01
 -5.55532540e-04  2.03250677e-01]
[-7.09746467e-05 -3.42835237e-04  1.72439592e-03 ...  2.47948722e-01
 -5.55532540e-04  2.03250677e-01]
[-7.09746467e-05 -3.42835237e-04  1.72439592e-03 ...  2.47948722e-01
 -5.

[-7.09746467e-05 -3.42835237e-04  1.72439592e-03 ...  2.47948722e-01
 -5.55532540e-04  2.03250677e-01]
[-7.09746467e-05 -3.42835237e-04  1.72439592e-03 ...  2.47948722e-01
 -5.55532540e-04  2.03250677e-01]
[-7.09746467e-05 -3.42835237e-04  1.72439592e-03 ...  2.47948722e-01
 -5.55532540e-04  2.03250677e-01]
[-7.09746467e-05 -3.42835237e-04  1.72439592e-03 ...  2.47948722e-01
 -5.55532540e-04  2.03250677e-01]
[-7.09746467e-05 -3.42835237e-04  1.72439592e-03 ...  2.47948722e-01
 -5.55532540e-04  2.03250677e-01]
[-7.09746467e-05 -3.42835237e-04  1.72439592e-03 ...  2.47948722e-01
 -5.55532540e-04  2.03250677e-01]
[-0.00160994 -0.00144061  0.00311345 ...  0.24728477 -0.00191584
  0.20493702]
[-7.09746467e-05 -3.42835237e-04  1.72439592e-03 ...  2.47948722e-01
 -5.55532540e-04  2.03250677e-01]
[-7.09746467e-05 -3.42835237e-04  1.72439592e-03 ...  2.47948722e-01
 -5.55532540e-04  2.03250677e-01]
[-7.09746467e-05 -3.42835237e-04  1.72439592e-03 ...  2.47948722e-01
 -5.55532540e-04  2.03250677

 -5.55532540e-04  2.03250677e-01]
[-7.09746467e-05 -3.42835237e-04  1.72439592e-03 ...  2.47948722e-01
 -5.55532540e-04  2.03250677e-01]
[-7.09746467e-05 -3.42835237e-04  1.72439592e-03 ...  2.47948722e-01
 -5.55532540e-04  2.03250677e-01]
[-0.00356477 -0.00365644 -0.00117757 ...  0.24793992 -0.00154843
  0.2009686 ]
[-7.09746467e-05 -3.42835237e-04  1.72439592e-03 ...  2.47948722e-01
 -5.55532540e-04  2.03250677e-01]
[-7.09746467e-05 -3.42835237e-04  1.72439592e-03 ...  2.47948722e-01
 -5.55532540e-04  2.03250677e-01]
[-7.09746467e-05 -3.42835237e-04  1.72439592e-03 ...  2.47948722e-01
 -5.55532540e-04  2.03250677e-01]
[-7.09746467e-05 -3.42835237e-04  1.72439592e-03 ...  2.47948722e-01
 -5.55532540e-04  2.03250677e-01]
[-7.09746467e-05 -3.42835237e-04  1.72439592e-03 ...  2.47948722e-01
 -5.55532540e-04  2.03250677e-01]
[-7.09746467e-05 -3.42835237e-04  1.72439592e-03 ...  2.47948722e-01
 -5.55532540e-04  2.03250677e-01]
[-7.09746467e-05 -3.42835237e-04  1.72439592e-03 ...  2.4794872

[-7.09746467e-05 -3.42835237e-04  1.72439592e-03 ...  2.47948722e-01
 -5.55532540e-04  2.03250677e-01]
[-7.09746467e-05 -3.42835237e-04  1.72439592e-03 ...  2.47948722e-01
 -5.55532540e-04  2.03250677e-01]
[-7.09746467e-05 -3.42835237e-04  1.72439592e-03 ...  2.47948722e-01
 -5.55532540e-04  2.03250677e-01]
[-7.09746467e-05 -3.42835237e-04  1.72439592e-03 ...  2.47948722e-01
 -5.55532540e-04  2.03250677e-01]
[-7.09746467e-05 -3.42835237e-04  1.72439592e-03 ...  2.47948722e-01
 -5.55532540e-04  2.03250677e-01]
[-7.09746467e-05 -3.42835237e-04  1.72439592e-03 ...  2.47948722e-01
 -5.55532540e-04  2.03250677e-01]
[-7.09746467e-05 -3.42835237e-04  1.72439592e-03 ...  2.47948722e-01
 -5.55532540e-04  2.03250677e-01]
[-7.09746467e-05 -3.42835237e-04  1.72439592e-03 ...  2.47948722e-01
 -5.55532540e-04  2.03250677e-01]
[-7.09746467e-05 -3.42835237e-04  1.72439592e-03 ...  2.47948722e-01
 -5.55532540e-04  2.03250677e-01]
[-7.09746467e-05 -3.42835237e-04  1.72439592e-03 ...  2.47948722e-01
 -5.

...

[-7.09746467e-05 -3.42835237e-04  1.72439592e-03 ...  2.47948722e-01
 -5.55532540e-04  2.03250677e-01]
[-7.09746467e-05 -3.42835237e-04  1.72439592e-03 ...  2.47948722e-01
 -5.55532540e-04  2.03250677e-01]
[-7.09746467e-05 -3.42835237e-04  1.72439592e-03 ...  2.47948722e-01
 -5.55532540e-04  2.03250677e-01]
[-7.09746467e-05 -3.42835237e-04  1.72439592e-03 ...  2.47948722e-01
 -5.55532540e-04  2.03250677e-01]
[-7.09746467e-05 -3.42835237e-04  1.72439592e-03 ...  2.47948722e-01
 -5.55532540e-04  2.03250677e-01]
[-7.09746467e-05 -3.42835237e-04  1.72439592e-03 ...  2.47948722e-01
 -5.55532540e-04  2.03250677e-01]
[-7.09746467e-05 -3.42835237e-04  1.72439592e-03 ...  2.47948722e-01
 -5.55532540e-04  2.03250677e-01]
[-7.09746467e-05 -3.42835237e-04  1.72439592e-03 ...  2.47948722e-01
 -5.55532540e-04  2.03250677e-01]
[-7.09746467e-05 -3.42835237e-04  1.72439592e-03 ...  2.47948722e-01
 -5.55532540e-04  2.03250677e-01]
[-7.09746467e-05 -3.42835237e-04  1.72439592e-03 ...  2.47948722e-01
 -5.

 -5.55532540e-04  2.03250677e-01]
[-7.09746467e-05 -3.42835237e-04  1.72439592e-03 ...  2.47948722e-01
 -5.55532540e-04  2.03250677e-01]
[-7.09746467e-05 -3.42835237e-04  1.72439592e-03 ...  2.47948722e-01
 -5.55532540e-04  2.03250677e-01]
[-7.09746467e-05 -3.42835237e-04  1.72439592e-03 ...  2.47948722e-01
 -5.55532540e-04  2.03250677e-01]
[-7.09746467e-05 -3.42835237e-04  1.72439592e-03 ...  2.47948722e-01
 -5.55532540e-04  2.03250677e-01]
[-7.09746467e-05 -3.42835237e-04  1.72439592e-03 ...  2.47948722e-01
 -5.55532540e-04  2.03250677e-01]
[-7.09746467e-05 -3.42835237e-04  1.72439592e-03 ...  2.47948722e-01
 -5.55532540e-04  2.03250677e-01]
[-7.09746467e-05 -3.42835237e-04  1.72439592e-03 ...  2.47948722e-01
 -5.55532540e-04  2.03250677e-01]
[ 0.0003775  -0.0007041   0.0016361  ...  0.24805135 -0.00067982
  0.2029263 ]
[-7.09746467e-05 -3.42835237e-04  1.72439592e-03 ...  2.47948722e-01
 -5.55532540e-04  2.03250677e-01]
[-7.09746467e-05 -3.42835237e-04  1.72439592e-03 ...  2.4794872

 -5.55532540e-04  2.03250677e-01]
[ 5.69969618e-04 -1.71297813e-03  1.57039934e-03 ...  2.47538247e-01
  3.89467756e-05  2.01626513e-01]
[-0.00029948 -0.00055779  0.00164952 ...  0.24765826 -0.00025949
  0.20339788]
[-7.09746467e-05 -3.42835237e-04  1.72439592e-03 ...  2.47948722e-01
 -5.55532540e-04  2.03250677e-01]
[-7.09746467e-05 -3.42835237e-04  1.72439592e-03 ...  2.47948722e-01
 -5.55532540e-04  2.03250677e-01]
[-7.09746467e-05 -3.42835237e-04  1.72439592e-03 ...  2.47948722e-01
 -5.55532540e-04  2.03250677e-01]
[-7.09746467e-05 -3.42835237e-04  1.72439592e-03 ...  2.47948722e-01
 -5.55532540e-04  2.03250677e-01]
[-7.09746467e-05 -3.42835237e-04  1.72439592e-03 ...  2.47948722e-01
 -5.55532540e-04  2.03250677e-01]
[-7.09746467e-05 -3.42835237e-04  1.72439592e-03 ...  2.47948722e-01
 -5.55532540e-04  2.03250677e-01]
[-7.09746467e-05 -3.42835237e-04  1.72439592e-03 ...  2.47948722e-01
 -5.55532540e-04  2.03250677e-01]
[-7.09746467e-05 -3.42835237e-04  1.72439592e-03 ...  2.4794872

[-7.09746467e-05 -3.42835237e-04  1.72439592e-03 ...  2.47948722e-01
 -5.55532540e-04  2.03250677e-01]
[-7.09746467e-05 -3.42835237e-04  1.72439592e-03 ...  2.47948722e-01
 -5.55532540e-04  2.03250677e-01]
[-7.09746467e-05 -3.42835237e-04  1.72439592e-03 ...  2.47948722e-01
 -5.55532540e-04  2.03250677e-01]
[-7.09746467e-05 -3.42835237e-04  1.72439592e-03 ...  2.47948722e-01
 -5.55532540e-04  2.03250677e-01]
[-7.09746467e-05 -3.42835237e-04  1.72439592e-03 ...  2.47948722e-01
 -5.55532540e-04  2.03250677e-01]
[-7.09746467e-05 -3.42835237e-04  1.72439592e-03 ...  2.47948722e-01
 -5.55532540e-04  2.03250677e-01]
[-7.09746467e-05 -3.42835237e-04  1.72439592e-03 ...  2.47948722e-01
 -5.55532540e-04  2.03250677e-01]
[-7.09746467e-05 -3.42835237e-04  1.72439592e-03 ...  2.47948722e-01
 -5.55532540e-04  2.03250677e-01]
[-7.09746467e-05 -3.42835237e-04  1.72439592e-03 ...  2.47948722e-01
 -5.55532540e-04  2.03250677e-01]
[-7.09746467e-05 -3.42835237e-04  1.72439592e-03 ...  2.47948722e-01
 -5.

[-7.09746467e-05 -3.42835237e-04  1.72439592e-03 ...  2.47948722e-01
 -5.55532540e-04  2.03250677e-01]
[-7.09746467e-05 -3.42835237e-04  1.72439592e-03 ...  2.47948722e-01
 -5.55532540e-04  2.03250677e-01]
[-7.09746467e-05 -3.42835237e-04  1.72439592e-03 ...  2.47948722e-01
 -5.55532540e-04  2.03250677e-01]
[-7.09746467e-05 -3.42835237e-04  1.72439592e-03 ...  2.47948722e-01
 -5.55532540e-04  2.03250677e-01]
[-7.09746467e-05 -3.42835237e-04  1.72439592e-03 ...  2.47948722e-01
 -5.55532540e-04  2.03250677e-01]
[-7.09746467e-05 -3.42835237e-04  1.72439592e-03 ...  2.47948722e-01
 -5.55532540e-04  2.03250677e-01]
[-7.09746467e-05 -3.42835237e-04  1.72439592e-03 ...  2.47948722e-01
 -5.55532540e-04  2.03250677e-01]
[-7.09746467e-05 -3.42835237e-04  1.72439592e-03 ...  2.47948722e-01
 -5.55532540e-04  2.03250677e-01]
[-7.09746467e-05 -3.42835237e-04  1.72439592e-03 ...  2.47948722e-01
 -5.55532540e-04  2.03250677e-01]
[-7.09746467e-05 -3.42835237e-04  1.72439592e-03 ...  2.47948722e-01
 -5.

In [14]:
import sklearn.metrics

# return a score
def compare_feature(target, given):
    return float(sklearn.metrics.pairwise.cosine_distances([np.array(target, dtype=float)], [np.array(given, dtype=float)]))
#     return float(np.linalg.norm(np.array(target, dtype=float) - np.array(given, dtype=float)))

def get_top_20(target):
    ret = []
    for index in range(MAX_OUTPUT):
        given = all_predict_test_des[index]
        score = compare_feature(target, given)
        item = str(score) + "___" + str(index)
        ret.append(item)
    return sorted(ret, key=functools.cmp_to_key(cmp))[:20]

# f = open("reverse_count_"+str(count_max_weight)+"unique"+str(uniqueness_max_weight)+".csv", "w")
f = open("reverse_nocount_tag_mlp.csv", "w")
f.write("Descritpion_ID,Top_20_Image_IDs\n")
for index in range(MAX_OUTPUT):
    target_desc = test_des_pca[index]
    twenty_image_list = get_top_20(target_desc)
    string_to_write = "" + str(index) + ".txt,"
    for name in twenty_image_list:
        file_name = name.split("___")[1] + ".jpg"+" "
        string_to_write += file_name
    string_to_write = string_to_write.strip() + "\n"
    print(string_to_write)
    f.write(string_to_write)

f.close()

0.txt,1842.jpg 15.jpg 111.jpg 120.jpg 121.jpg 184.jpg 250.jpg 281.jpg 311.jpg 381.jpg 388.jpg 445.jpg 452.jpg 607.jpg 649.jpg 700.jpg 763.jpg 768.jpg 784.jpg 821.jpg

1.txt,833.jpg 589.jpg 1156.jpg 1341.jpg 1417.jpg 1529.jpg 435.jpg 562.jpg 948.jpg 1071.jpg 753.jpg 1337.jpg 1929.jpg 758.jpg 1907.jpg 809.jpg 542.jpg 956.jpg 687.jpg 1753.jpg

2.txt,1661.jpg 43.jpg 97.jpg 272.jpg 303.jpg 332.jpg 408.jpg 545.jpg 598.jpg 625.jpg 662.jpg 755.jpg 790.jpg 915.jpg 980.jpg 990.jpg 993.jpg 1060.jpg 1225.jpg 1338.jpg

3.txt,15.jpg 111.jpg 120.jpg 121.jpg 184.jpg 250.jpg 281.jpg 311.jpg 381.jpg 388.jpg 445.jpg 452.jpg 607.jpg 649.jpg 700.jpg 763.jpg 768.jpg 784.jpg 821.jpg 860.jpg

4.txt,1665.jpg 1124.jpg 1842.jpg 15.jpg 111.jpg 120.jpg 121.jpg 184.jpg 250.jpg 281.jpg 311.jpg 381.jpg 388.jpg 445.jpg 452.jpg 607.jpg 649.jpg 700.jpg 763.jpg 768.jpg

5.txt,46.jpg 51.jpg 274.jpg 330.jpg 334.jpg 354.jpg 492.jpg 526.jpg 554.jpg 614.jpg 699.jpg 703.jpg 771.jpg 814.jpg 887.jpg 1045.jpg 1118.jpg 1287.jpg 13

48.txt,1124.jpg 15.jpg 111.jpg 120.jpg 121.jpg 184.jpg 250.jpg 281.jpg 311.jpg 381.jpg 388.jpg 445.jpg 452.jpg 607.jpg 649.jpg 700.jpg 763.jpg 768.jpg 784.jpg 821.jpg

49.txt,63.jpg 559.jpg 646.jpg 690.jpg 761.jpg 892.jpg 1235.jpg 1396.jpg 1739.jpg 180.jpg 306.jpg 319.jpg 349.jpg 398.jpg 443.jpg 509.jpg 603.jpg 680.jpg 697.jpg 748.jpg

50.txt,494.jpg 384.jpg 10.jpg 54.jpg 213.jpg 560.jpg 577.jpg 886.jpg 932.jpg 1056.jpg 1123.jpg 1190.jpg 1210.jpg 1290.jpg 1362.jpg 1363.jpg 1448.jpg 1469.jpg 1534.jpg 1680.jpg

51.txt,475.jpg 641.jpg 695.jpg 909.jpg 1517.jpg 26.jpg 78.jpg 94.jpg 172.jpg 267.jpg 471.jpg 514.jpg 825.jpg 989.jpg 1132.jpg 1145.jpg 1207.jpg 1215.jpg 1373.jpg 1404.jpg

52.txt,32.jpg 287.jpg 295.jpg 407.jpg 1137.jpg 1846.jpg 346.jpg 745.jpg 1926.jpg 270.jpg 1877.jpg 1798.jpg 1598.jpg 1921.jpg 307.jpg 308.jpg 405.jpg 466.jpg 1106.jpg 1167.jpg

53.txt,229.jpg 254.jpg 1661.jpg 43.jpg 97.jpg 272.jpg 303.jpg 332.jpg 408.jpg 545.jpg 598.jpg 625.jpg 662.jpg 755.jpg 790.jpg 915.jpg 980

96.txt,159.jpg 372.jpg 394.jpg 497.jpg 561.jpg 720.jpg 813.jpg 904.jpg 953.jpg 1023.jpg 1073.jpg 1084.jpg 1107.jpg 1617.jpg 1695.jpg 1717.jpg 1950.jpg 1955.jpg 1987.jpg 1713.jpg

97.txt,399.jpg 962.jpg 1739.jpg 180.jpg 306.jpg 319.jpg 349.jpg 398.jpg 443.jpg 509.jpg 603.jpg 680.jpg 697.jpg 748.jpg 783.jpg 867.jpg 901.jpg 938.jpg 1008.jpg 1164.jpg

98.txt,1665.jpg 15.jpg 111.jpg 120.jpg 121.jpg 184.jpg 250.jpg 281.jpg 311.jpg 381.jpg 388.jpg 445.jpg 452.jpg 607.jpg 649.jpg 700.jpg 763.jpg 768.jpg 784.jpg 821.jpg

99.txt,1038.jpg 1150.jpg 1579.jpg 1640.jpg 1685.jpg 1764.jpg 1859.jpg 1744.jpg 337.jpg 1124.jpg 15.jpg 111.jpg 120.jpg 121.jpg 184.jpg 250.jpg 281.jpg 311.jpg 381.jpg 388.jpg

100.txt,1842.jpg 1124.jpg 15.jpg 111.jpg 120.jpg 121.jpg 184.jpg 250.jpg 281.jpg 311.jpg 381.jpg 388.jpg 445.jpg 452.jpg 607.jpg 649.jpg 700.jpg 763.jpg 768.jpg 784.jpg

101.txt,513.jpg 692.jpg 694.jpg 1001.jpg 1079.jpg 1300.jpg 1537.jpg 1563.jpg 1869.jpg 162.jpg 233.jpg 383.jpg 639.jpg 1100.jpg 1238.jpg 

144.txt,653.jpg 729.jpg 926.jpg 1200.jpg 1204.jpg 1342.jpg 1430.jpg 307.jpg 308.jpg 405.jpg 466.jpg 1106.jpg 1167.jpg 1260.jpg 1304.jpg 1728.jpg 1827.jpg 1171.jpg 1077.jpg 1409.jpg

145.txt,513.jpg 692.jpg 694.jpg 1001.jpg 1079.jpg 1300.jpg 1537.jpg 1563.jpg 1869.jpg 162.jpg 233.jpg 383.jpg 639.jpg 1100.jpg 1238.jpg 1276.jpg 1718.jpg 1975.jpg 1983.jpg 395.jpg

146.txt,1842.jpg 15.jpg 111.jpg 120.jpg 121.jpg 184.jpg 250.jpg 281.jpg 311.jpg 381.jpg 388.jpg 445.jpg 452.jpg 607.jpg 649.jpg 700.jpg 763.jpg 768.jpg 784.jpg 821.jpg

147.txt,1124.jpg 15.jpg 111.jpg 120.jpg 121.jpg 184.jpg 250.jpg 281.jpg 311.jpg 381.jpg 388.jpg 445.jpg 452.jpg 607.jpg 649.jpg 700.jpg 763.jpg 768.jpg 784.jpg 821.jpg

148.txt,1743.jpg 1576.jpg 12.jpg 28.jpg 36.jpg 64.jpg 70.jpg 87.jpg 101.jpg 128.jpg 138.jpg 141.jpg 147.jpg 151.jpg 152.jpg 194.jpg 210.jpg 214.jpg 256.jpg 268.jpg

149.txt,46.jpg 51.jpg 274.jpg 330.jpg 334.jpg 354.jpg 492.jpg 526.jpg 554.jpg 614.jpg 699.jpg 703.jpg 771.jpg 814.jpg 887.jpg 1045.jpg

192.txt,286.jpg 885.jpg 1039.jpg 1134.jpg 1143.jpg 169.jpg 331.jpg 1196.jpg 14.jpg 815.jpg 1110.jpg 1726.jpg 1526.jpg 495.jpg 515.jpg 1456.jpg 553.jpg 296.jpg 1989.jpg 154.jpg

193.txt,384.jpg 494.jpg 1832.jpg 10.jpg 54.jpg 213.jpg 560.jpg 577.jpg 886.jpg 932.jpg 1056.jpg 1123.jpg 1190.jpg 1210.jpg 1290.jpg 1362.jpg 1363.jpg 1448.jpg 1469.jpg 1534.jpg

194.txt,1598.jpg 1921.jpg 416.jpg 2.jpg 31.jpg 37.jpg 52.jpg 77.jpg 620.jpg 670.jpg 734.jpg 880.jpg 914.jpg 972.jpg 1098.jpg 1127.jpg 1236.jpg 1318.jpg 1385.jpg 1415.jpg

195.txt,493.jpg 434.jpg 88.jpg 116.jpg 140.jpg 375.jpg 637.jpg 711.jpg 862.jpg 902.jpg 936.jpg 939.jpg 995.jpg 1050.jpg 1069.jpg 1105.jpg 1165.jpg 1180.jpg 1263.jpg 1480.jpg

196.txt,806.jpg 861.jpg 436.jpg 1072.jpg 318.jpg 621.jpg 793.jpg 911.jpg 206.jpg 1388.jpg 34.jpg 1730.jpg 1658.jpg 532.jpg 597.jpg 1892.jpg 483.jpg 895.jpg 461.jpg 610.jpg

197.txt,1598.jpg 1921.jpg 2.jpg 31.jpg 37.jpg 52.jpg 77.jpg 620.jpg 670.jpg 734.jpg 880.jpg 914.jpg 972.jpg 1098.jpg 1127.jpg 

...

1805.txt,1124.jpg 15.jpg 111.jpg 120.jpg 121.jpg 184.jpg 250.jpg 281.jpg 311.jpg 381.jpg 388.jpg 445.jpg 452.jpg 607.jpg 649.jpg 700.jpg 763.jpg 768.jpg 784.jpg 821.jpg

1806.txt,399.jpg 962.jpg 180.jpg 306.jpg 319.jpg 349.jpg 398.jpg 443.jpg 509.jpg 603.jpg 680.jpg 697.jpg 748.jpg 783.jpg 867.jpg 901.jpg 938.jpg 1008.jpg 1164.jpg 1211.jpg

1807.txt,243.jpg 1140.jpg 1457.jpg 1755.jpg 75.jpg 397.jpg 654.jpg 799.jpg 800.jpg 1035.jpg 1486.jpg 1513.jpg 1865.jpg 1922.jpg 836.jpg 1857.jpg 1315.jpg 1.jpg 53.jpg 285.jpg

1808.txt,1124.jpg 1842.jpg 15.jpg 111.jpg 120.jpg 121.jpg 184.jpg 250.jpg 281.jpg 311.jpg 381.jpg 388.jpg 445.jpg 452.jpg 607.jpg 649.jpg 700.jpg 763.jpg 768.jpg 784.jpg

1809.txt,193.jpg 1585.jpg 1938.jpg 1038.jpg 1150.jpg 1579.jpg 1640.jpg 1685.jpg 1764.jpg 1859.jpg 1744.jpg 657.jpg 714.jpg 1281.jpg 125.jpg 839.jpg 444.jpg 230.jpg 178.jpg 164.jpg

1810.txt,447.jpg 1104.jpg 803.jpg 1387.jpg 1927.jpg 1834.jpg 195.jpg 241.jpg 277.jpg 283.jpg 327.jpg 472.jpg 477.jpg 551.jpg 558.

1852.txt,65.jpg 613.jpg 801.jpg 875.jpg 1091.jpg 1295.jpg 1372.jpg 1485.jpg 1501.jpg 1590.jpg 1619.jpg 1697.jpg 1784.jpg 1861.jpg 1908.jpg 350.jpg 1306.jpg 851.jpg 751.jpg 1935.jpg

1853.txt,1536.jpg 79.jpg 922.jpg 1446.jpg 1181.jpg 1273.jpg 519.jpg 1103.jpg 165.jpg 946.jpg 1672.jpg 8.jpg 19.jpg 223.jpg 338.jpg 785.jpg 1051.jpg 1149.jpg 1176.jpg 1461.jpg

1854.txt,65.jpg 613.jpg 801.jpg 875.jpg 1091.jpg 1295.jpg 1372.jpg 1485.jpg 1501.jpg 1590.jpg 1619.jpg 1697.jpg 1784.jpg 1861.jpg 1908.jpg 350.jpg 1306.jpg 851.jpg 751.jpg 1935.jpg

1855.txt,1160.jpg 1942.jpg 470.jpg 713.jpg 746.jpg 963.jpg 988.jpg 1578.jpg 1951.jpg 658.jpg 165.jpg 946.jpg 1672.jpg 251.jpg 95.jpg 484.jpg 1566.jpg 1868.jpg 1252.jpg 1257.jpg

1856.txt,15.jpg 111.jpg 120.jpg 121.jpg 184.jpg 250.jpg 281.jpg 311.jpg 381.jpg 388.jpg 445.jpg 452.jpg 607.jpg 649.jpg 700.jpg 763.jpg 768.jpg 784.jpg 821.jpg 860.jpg

1857.txt,15.jpg 111.jpg 120.jpg 121.jpg 184.jpg 250.jpg 281.jpg 311.jpg 381.jpg 388.jpg 445.jpg 452.jpg 607.jpg 6

1899.txt,55.jpg 153.jpg 157.jpg 224.jpg 371.jpg 739.jpg 843.jpg 893.jpg 943.jpg 950.jpg 981.jpg 1024.jpg 1547.jpg 1662.jpg 1771.jpg 1960.jpg 192.jpg 290.jpg 609.jpg 633.jpg

1900.txt,1744.jpg 1038.jpg 1150.jpg 1579.jpg 1640.jpg 1685.jpg 1764.jpg 1859.jpg 337.jpg 15.jpg 111.jpg 120.jpg 121.jpg 184.jpg 250.jpg 281.jpg 311.jpg 381.jpg 388.jpg 445.jpg

1901.txt,1665.jpg 15.jpg 111.jpg 120.jpg 121.jpg 184.jpg 250.jpg 281.jpg 311.jpg 381.jpg 388.jpg 445.jpg 452.jpg 607.jpg 649.jpg 700.jpg 763.jpg 768.jpg 784.jpg 821.jpg

1902.txt,109.jpg 757.jpg 804.jpg 869.jpg 997.jpg 1054.jpg 1669.jpg 1741.jpg 1911.jpg 1973.jpg 7.jpg 57.jpg 237.jpg 1428.jpg 1650.jpg 355.jpg 369.jpg 815.jpg 1880.jpg 181.jpg

1903.txt,136.jpg 409.jpg 448.jpg 481.jpg 528.jpg 627.jpg 685.jpg 689.jpg 756.jpg 987.jpg 1085.jpg 1155.jpg 1239.jpg 1249.jpg 1270.jpg 1285.jpg 1488.jpg 1694.jpg 1900.jpg 1961.jpg

1904.txt,589.jpg 1156.jpg 1341.jpg 1417.jpg 1529.jpg 833.jpg 435.jpg 562.jpg 948.jpg 1071.jpg 753.jpg 1337.jpg 1929.jpg 758.

1946.txt,384.jpg 10.jpg 54.jpg 213.jpg 560.jpg 577.jpg 886.jpg 932.jpg 1056.jpg 1123.jpg 1190.jpg 1210.jpg 1290.jpg 1362.jpg 1363.jpg 1448.jpg 1469.jpg 1534.jpg 1680.jpg 1703.jpg

1947.txt,30.jpg 538.jpg 1688.jpg 1810.jpg 572.jpg 738.jpg 1507.jpg 753.jpg 1196.jpg 1347.jpg 1173.jpg 1740.jpg 392.jpg 182.jpg 984.jpg 197.jpg 1880.jpg 15.jpg 111.jpg 120.jpg

1948.txt,1744.jpg 1038.jpg 1150.jpg 1579.jpg 1640.jpg 1685.jpg 1764.jpg 1859.jpg 1576.jpg 12.jpg 28.jpg 36.jpg 64.jpg 70.jpg 87.jpg 101.jpg 128.jpg 138.jpg 141.jpg 147.jpg

1949.txt,195.jpg 241.jpg 277.jpg 283.jpg 327.jpg 472.jpg 477.jpg 551.jpg 558.jpg 664.jpg 955.jpg 1089.jpg 1244.jpg 1544.jpg 1583.jpg 1604.jpg 1747.jpg 1766.jpg 1829.jpg 1947.jpg

1950.txt,134.jpg 266.jpg 419.jpg 719.jpg 1346.jpg 1397.jpg 1557.jpg 1622.jpg 1948.jpg 69.jpg 1037.jpg 1261.jpg 1543.jpg 1817.jpg 1847.jpg 1918.jpg 3.jpg 103.jpg 245.jpg 823.jpg

1951.txt,1665.jpg 1842.jpg 15.jpg 111.jpg 120.jpg 121.jpg 184.jpg 250.jpg 281.jpg 311.jpg 381.jpg 388.jpg 445.jpg 

1993.txt,65.jpg 613.jpg 801.jpg 875.jpg 1091.jpg 1295.jpg 1372.jpg 1485.jpg 1501.jpg 1590.jpg 1619.jpg 1697.jpg 1784.jpg 1861.jpg 1908.jpg 350.jpg 1306.jpg 851.jpg 751.jpg 1935.jpg

1994.txt,1598.jpg 1921.jpg 2.jpg 31.jpg 37.jpg 52.jpg 77.jpg 620.jpg 670.jpg 734.jpg 880.jpg 914.jpg 972.jpg 1098.jpg 1127.jpg 1236.jpg 1318.jpg 1385.jpg 1415.jpg 1433.jpg

1995.txt,1124.jpg 15.jpg 111.jpg 120.jpg 121.jpg 184.jpg 250.jpg 281.jpg 311.jpg 381.jpg 388.jpg 445.jpg 452.jpg 607.jpg 649.jpg 700.jpg 763.jpg 768.jpg 784.jpg 821.jpg

1996.txt,180.jpg 306.jpg 319.jpg 349.jpg 398.jpg 443.jpg 509.jpg 603.jpg 680.jpg 697.jpg 748.jpg 783.jpg 867.jpg 901.jpg 938.jpg 1008.jpg 1164.jpg 1211.jpg 1392.jpg 1440.jpg

1997.txt,1026.jpg 326.jpg 701.jpg 1138.jpg 1988.jpg 104.jpg 1638.jpg 58.jpg 293.jpg 12.jpg 28.jpg 36.jpg 64.jpg 70.jpg 87.jpg 101.jpg 128.jpg 138.jpg 141.jpg 147.jpg

1998.txt,806.jpg 861.jpg 436.jpg 1072.jpg 318.jpg 621.jpg 793.jpg 911.jpg 206.jpg 1388.jpg 34.jpg 1842.jpg 15.jpg 111.jpg 120.jpg 121